In [1]:
import sys
import os

import pymysql as sql

## 하위 폴더에 있는 패키지 로딩을 위한 밑작업
SEP       = os.path.sep
MISC_PATH = SEP.join(os.getcwd().split(SEP)[:-2])
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-4])
sys.path.append(MISC_PATH)

from misc.config import *

[INFO] config.json 파일을 로딩합니다.
[INFO] ports.json 파일을 로딩합니다.


In [2]:
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user
passwd = CONFIGS.sql_passwd

In [3]:
conn   = sql.connect(host   =   host, user = user, port = port,
                     passwd = passwd, db   = 'study')
cursor = conn.cursor()

In [4]:
def print_fetch(text: str, results: tuple):
    
    print(f'[{text}]')
    for result in results: print(result)
    print('\n')
    

def select(table_name: str, text: str = None, column: str = '*', 
           distinct: bool = False, cond: str = None,
           order: str = None, limit_k: str = None):
    
    
    query = f'select distinct {column} from {table_name}' if distinct \
            else f'select {column} from {table_name}'
    
    if cond != None: query += f' WHERE {cond}'
    
    if order != None: query += f' ORDER BY {order}'
    
    if limit_k != None: query += f' LIMIT {limit_k}'
    
    print(f'[query] {query};\n')
    
    cursor.execute(query)
    if text != None: print_fetch(text, cursor.fetchall())
           

In [5]:
cursor.execute('show tables')
for result in cursor.fetchall(): print(result)
print('\n')

('department',)
('dept_',)
('employee',)
('employee_',)
('tCity',)
('tStaff',)




In [6]:
select(column = 'column_name', text = 'employee table columns',
      table_name = 'information_schema.columns',
      cond = 'table_name = "employee"')

select(column = 'column_name', text = 'department table columns',
      table_name = 'information_schema.columns',
      cond = 'table_name = "department"')

[query] select column_name from information_schema.columns WHERE table_name = "employee";

[employee table columns]
('eno',)
('ename',)
('job',)
('manager',)
('hiredate',)
('salary',)
('commission',)
('dno',)


[query] select column_name from information_schema.columns WHERE table_name = "department";

[department table columns]
('dno',)
('dname',)
('loc',)




In [7]:
select(text = 'whole datas in employee table'  ,   table_name = 'employee')

select(text = 'whole datas in department table', table_name = 'department')

[query] select * from employee;

[whole datas in employee table]
('101', 'e1', 'section', '102', datetime.datetime(2020, 1, 1, 0, 0), 100, 100, '10')
('102', 'e2', 'chief', '103', datetime.datetime(2020, 1, 2, 0, 0), 200, None, '10')
('103', 'e3', 'senior', '105', datetime.datetime(2020, 1, 3, 0, 0), 2500, 200, '30')
('104', 'e4', 'section', '107', datetime.datetime(2020, 1, 4, 0, 0), 400, None, '30')
('105', 'e5', 'chief', '101', datetime.datetime(2020, 1, 5, 0, 0), 450, 100, '20')
('106', 'e6', 'senior', '108', datetime.datetime(2020, 1, 6, 0, 0), 500, None, '10')
('107', 'e7', 'section', '109', datetime.datetime(2020, 1, 7, 0, 0), 520, 100, '30')
('108', 'e8', 'chief', '103', datetime.datetime(2020, 1, 8, 0, 0), 670, None, '20')
('109', 'e9', 'senior', '103', datetime.datetime(2020, 1, 9, 0, 0), 800, 100, '10')
('110', 'e10', 'section', '103', datetime.datetime(2020, 1, 10, 0, 0), 120, None, '20')
('111', 'e11', 'chief', '104', datetime.datetime(2020, 1, 11, 0, 0), 130, 0, '10')
('1

 #### **[Q #001]. employee 테이블에서 사원명과 봉급 * 12를 조회하라.**

In [8]:
select(text   = 'A #001', table_name = 'employee',
       column = 'ename, salary * 12')

[query] select ename, salary * 12 from employee;

[A #001]
('e1', 1200)
('e2', 2400)
('e3', 30000)
('e4', 4800)
('e5', 5400)
('e6', 6000)
('e7', 6240)
('e8', 8040)
('e9', 9600)
('e10', 1440)
('e11', 1560)
('e12', 2640)
('e13', 6240)




#### **[Q #002]. employee 테이블에서 사원번호 110번의 이름과 부서번호.**

In [9]:
select(text = 'A #002', table_name = 'employee',
       column = 'ename, dno', cond = 'eno=110')

select(text = 'A #002 REF', table_name = 'employee',
       column = 'ename, dno, eno', cond = 'eno = 110')

[query] select ename, dno from employee WHERE eno=110;

[A #002]
('e10', '20')


[query] select ename, dno, eno from employee WHERE eno = 110;

[A #002 REF]
('e10', '20', '110')




#### **[Q #003]. employee 테이블에서 부서번호가 20번이고 월급이 400이상인 사원의 이름과 직책**

In [10]:
select(text = 'A #003.', column = 'ename, job', table_name = 'employee',
       cond = 'dno = 20 and salary >= 400')

select(text = 'A #003 REF', table_name = 'employee',
       column = 'ename, job, dno, salary', cond = 'dno = 20 and salary >= 400')

[query] select ename, job from employee WHERE dno = 20 and salary >= 400;

[A #003.]
('e5', 'chief')
('e8', 'chief')


[query] select ename, job, dno, salary from employee WHERE dno = 20 and salary >= 400;

[A #003 REF]
('e5', 'chief', '20', 450)
('e8', 'chief', '20', 670)




#### **[Q #004]. 입사일이 2020년 1월 10일에서 2020년 1월 12일 사이인 회원의 이름과 직책**

In [11]:
select(text = 'A #004', table_name = 'employee',
       cond = 'hiredate between 20200110 and 20200112')

[query] select * from employee WHERE hiredate between 20200110 and 20200112;

[A #004]
('110', 'e10', 'section', '103', datetime.datetime(2020, 1, 10, 0, 0), 120, None, '20')
('111', 'e11', 'chief', '104', datetime.datetime(2020, 1, 11, 0, 0), 130, 0, '10')
('112', 'e12', 'senior', '101', datetime.datetime(2020, 1, 12, 0, 0), 220, None, '30')




#### **[Q #005]. employee 테이블에서 부서번호가 10 이거나 30인 회원의 이름과 직책**

In [12]:
select(text = 'A #005', table_name = 'employee',
       column = 'ename, job', cond = 'dno in (10, 30)')

select(text = 'A #005 REF', table_name = 'employee',
       column = 'ename, job, dno', cond = 'dno in (10, 30)')

[query] select ename, job from employee WHERE dno in (10, 30);

[A #005]
('e1', 'section')
('e2', 'chief')
('e3', 'senior')
('e4', 'section')
('e6', 'senior')
('e7', 'section')
('e9', 'senior')
('e11', 'chief')
('e12', 'senior')
('e13', 'section')


[query] select ename, job, dno from employee WHERE dno in (10, 30);

[A #005 REF]
('e1', 'section', '10')
('e2', 'chief', '10')
('e3', 'senior', '30')
('e4', 'section', '30')
('e6', 'senior', '10')
('e7', 'section', '30')
('e9', 'senior', '10')
('e11', 'chief', '10')
('e12', 'senior', '30')
('e13', 'section', '10')




#### **[Q #006]. employee 테이블에서 보너스가 null값인 직원의 모든 속성**

In [13]:
select(text = 'A #006', table_name = 'employee',
       cond = 'commission is Null')

[query] select * from employee WHERE commission is Null;

[A #006]
('102', 'e2', 'chief', '103', datetime.datetime(2020, 1, 2, 0, 0), 200, None, '10')
('104', 'e4', 'section', '107', datetime.datetime(2020, 1, 4, 0, 0), 400, None, '30')
('106', 'e6', 'senior', '108', datetime.datetime(2020, 1, 6, 0, 0), 500, None, '10')
('108', 'e8', 'chief', '103', datetime.datetime(2020, 1, 8, 0, 0), 670, None, '20')
('110', 'e10', 'section', '103', datetime.datetime(2020, 1, 10, 0, 0), 120, None, '20')
('112', 'e12', 'senior', '101', datetime.datetime(2020, 1, 12, 0, 0), 220, None, '30')




#### **[Q #007]. employee 테이블에서 직책에 ie가 들어가는 직원의 이름, 직책, 부서번호**

In [14]:
select(text = 'A #007', table_name = 'employee',
       column = 'ename, job, dno', cond = 'job like "%ie%"')

[query] select ename, job, dno from employee WHERE job like "%ie%";

[A #007]
('e2', 'chief', '10')
('e5', 'chief', '20')
('e8', 'chief', '20')
('e11', 'chief', '10')




#### **[Q #008]. 월급 기준 내림차순으로 사원들의 이름, 월급, 부서번호.**
- 단, 월급이 같은 경우 부서번호를 오름차 순으로 재정렬

In [15]:
select(text = 'A #008', table_name = 'employee',
       column = 'ename, salary, dno', order = 'salary desc, dno')

[query] select ename, salary, dno from employee ORDER BY salary desc, dno;

[A #008]
('e3', 2500, '30')
('e9', 800, '10')
('e8', 670, '20')
('e13', 520, '10')
('e7', 520, '30')
('e6', 500, '10')
('e5', 450, '20')
('e4', 400, '30')
('e12', 220, '30')
('e2', 200, '10')
('e11', 130, '10')
('e10', 120, '20')
('e1', 100, '10')




#### **[Q #009]. employee 테이블의 모든 데이터.**

In [16]:
select(text = 'A #009', table_name = 'employee')

[query] select * from employee;

[A #009]
('101', 'e1', 'section', '102', datetime.datetime(2020, 1, 1, 0, 0), 100, 100, '10')
('102', 'e2', 'chief', '103', datetime.datetime(2020, 1, 2, 0, 0), 200, None, '10')
('103', 'e3', 'senior', '105', datetime.datetime(2020, 1, 3, 0, 0), 2500, 200, '30')
('104', 'e4', 'section', '107', datetime.datetime(2020, 1, 4, 0, 0), 400, None, '30')
('105', 'e5', 'chief', '101', datetime.datetime(2020, 1, 5, 0, 0), 450, 100, '20')
('106', 'e6', 'senior', '108', datetime.datetime(2020, 1, 6, 0, 0), 500, None, '10')
('107', 'e7', 'section', '109', datetime.datetime(2020, 1, 7, 0, 0), 520, 100, '30')
('108', 'e8', 'chief', '103', datetime.datetime(2020, 1, 8, 0, 0), 670, None, '20')
('109', 'e9', 'senior', '103', datetime.datetime(2020, 1, 9, 0, 0), 800, 100, '10')
('110', 'e10', 'section', '103', datetime.datetime(2020, 1, 10, 0, 0), 120, None, '20')
('111', 'e11', 'chief', '104', datetime.datetime(2020, 1, 11, 0, 0), 130, 0, '10')
('112', 'e12', 'senior', '

#### **[Q #010]. employee 테이블에서 사원명과 입사일.**

In [17]:
select(text   = 'A #010', table_name = 'employee',
       column = 'ename, hiredate')

[query] select ename, hiredate from employee;

[A #010]
('e1', datetime.datetime(2020, 1, 1, 0, 0))
('e2', datetime.datetime(2020, 1, 2, 0, 0))
('e3', datetime.datetime(2020, 1, 3, 0, 0))
('e4', datetime.datetime(2020, 1, 4, 0, 0))
('e5', datetime.datetime(2020, 1, 5, 0, 0))
('e6', datetime.datetime(2020, 1, 6, 0, 0))
('e7', datetime.datetime(2020, 1, 7, 0, 0))
('e8', datetime.datetime(2020, 1, 8, 0, 0))
('e9', datetime.datetime(2020, 1, 9, 0, 0))
('e10', datetime.datetime(2020, 1, 10, 0, 0))
('e11', datetime.datetime(2020, 1, 11, 0, 0))
('e12', datetime.datetime(2020, 1, 12, 0, 0))
('e13', datetime.datetime(2020, 1, 13, 0, 0))




#### **[Q #011]. employee 테이블에서 직책의 목록.**

In [18]:
select(text  = 'A #011', table_name = 'employee',
      distinct = True, column = 'job')

[query] select distinct job from employee;

[A #011]
('section',)
('chief',)
('senior',)




#### **[Q #012]. employee 테이블에서 부서번호가 10인 직원.**

In [19]:
select(text = 'A #012', table_name = 'employee',
       cond = 'dno = 10')

[query] select * from employee WHERE dno = 10;

[A #012]
('101', 'e1', 'section', '102', datetime.datetime(2020, 1, 1, 0, 0), 100, 100, '10')
('102', 'e2', 'chief', '103', datetime.datetime(2020, 1, 2, 0, 0), 200, None, '10')
('106', 'e6', 'senior', '108', datetime.datetime(2020, 1, 6, 0, 0), 500, None, '10')
('109', 'e9', 'senior', '103', datetime.datetime(2020, 1, 9, 0, 0), 800, 100, '10')
('111', 'e11', 'chief', '104', datetime.datetime(2020, 1, 11, 0, 0), 130, 0, '10')
('113', 'e13', 'section', '102', datetime.datetime(2020, 1, 13, 0, 0), 520, 100, '10')




#### **[Q #013]. employee 테이블에서 월급이 2500이상이 되는 직원.**

In [20]:
select(text = 'A #013', table_name = 'employee',
       cond = 'salary >= 2500')

[query] select * from employee WHERE salary >= 2500;

[A #013]
('103', 'e3', 'senior', '105', datetime.datetime(2020, 1, 3, 0, 0), 2500, 200, '30')




#### **[Q #014]. employee 테이블에서 이름이 e10인 직원.**

In [21]:
select(text = 'A #014', table_name = 'employee', cond = 'ename = "e10"')

[query] select * from employee WHERE ename = "e10";

[A #014]
('110', 'e10', 'section', '103', datetime.datetime(2020, 1, 10, 0, 0), 120, None, '20')




#### **[Q #015]. 직책이 S로 시작하는 사원의 사원 번호와 이름.**

In [22]:
select(text   = 'A #015', table_name = 'employee',
       column = 'eno, ename', cond = 'job like "S%"')

select(text = 'A #015 REF', table_name = 'employee',
      column = 'eno, ename, job', cond = 'job like "S%"')

[query] select eno, ename from employee WHERE job like "S%";

[A #015]
('101', 'e1')
('103', 'e3')
('104', 'e4')
('106', 'e6')
('107', 'e7')
('109', 'e9')
('110', 'e10')
('112', 'e12')
('113', 'e13')


[query] select eno, ename, job from employee WHERE job like "S%";

[A #015 REF]
('101', 'e1', 'section')
('103', 'e3', 'senior')
('104', 'e4', 'section')
('106', 'e6', 'senior')
('107', 'e7', 'section')
('109', 'e9', 'senior')
('110', 'e10', 'section')
('112', 'e12', 'senior')
('113', 'e13', 'section')




#### **[Q #016]. 사원 직책에 e가 포함된 사원의 사원번호와 이름.**

In [23]:
select(text   = 'A #016', table_name = 'employee',
       column = 'eno, ename', cond = 'job like "%e%"')

select(text = 'A #016 REF', table_name = 'employee',
      column = 'eno, ename, job', cond = 'job like "%e%"')

[query] select eno, ename from employee WHERE job like "%e%";

[A #016]
('101', 'e1')
('102', 'e2')
('103', 'e3')
('104', 'e4')
('105', 'e5')
('106', 'e6')
('107', 'e7')
('108', 'e8')
('109', 'e9')
('110', 'e10')
('111', 'e11')
('112', 'e12')
('113', 'e13')


[query] select eno, ename, job from employee WHERE job like "%e%";

[A #016 REF]
('101', 'e1', 'section')
('102', 'e2', 'chief')
('103', 'e3', 'senior')
('104', 'e4', 'section')
('105', 'e5', 'chief')
('106', 'e6', 'senior')
('107', 'e7', 'section')
('108', 'e8', 'chief')
('109', 'e9', 'senior')
('110', 'e10', 'section')
('111', 'e11', 'chief')
('112', 'e12', 'senior')
('113', 'e13', 'section')




#### **[Q #017]. 월급이 200, 400, 2500인 직원의 사번, 이름,보너스.**

In [24]:
select(text   = 'A #017', table_name = 'employee',
       column = 'eno, ename, commission', cond = 'salary in (200, 400, 2500)')

select(text = 'A #017 REF', table_name = 'employee',
      column = 'eno, ename, commission, salary', cond = 'salary in (200, 400, 2500)')

[query] select eno, ename, commission from employee WHERE salary in (200, 400, 2500);

[A #017]
('102', 'e2', None)
('103', 'e3', 200)
('104', 'e4', None)


[query] select eno, ename, commission, salary from employee WHERE salary in (200, 400, 2500);

[A #017 REF]
('102', 'e2', None, 200)
('103', 'e3', 200, 2500)
('104', 'e4', None, 400)




#### **[Q #018]. 직책이 chief이고 부서번호가 30번인 직원의 이름, 사번, 직급, 부서번호.**

In [25]:
select(text   = 'A #018', table_name = 'employee',
       column = 'ename, eno, job, dno', cond = 'job = "chief" and dno = 30')

select(text = 'A #018 REF', table_name = 'employee',
       column = 'ename, eno, job, dno', cond = 'job = "chief"')


[query] select ename, eno, job, dno from employee WHERE job = "chief" and dno = 30;

[A #018]


[query] select ename, eno, job, dno from employee WHERE job = "chief";

[A #018 REF]
('e2', '102', 'chief', '10')
('e5', '105', 'chief', '20')
('e8', '108', 'chief', '20')
('e11', '111', 'chief', '10')




#### **[Q #019]. 부서번호가 30이 아닌 직원의 사번, 이름, 부서번호.**

In [26]:
select(text   = 'A #019', table_name = 'employee',
       column = 'eno, ename, dno', cond = 'dno <> 30')

[query] select eno, ename, dno from employee WHERE dno <> 30;

[A #019]
('101', 'e1', '10')
('102', 'e2', '10')
('105', 'e5', '20')
('106', 'e6', '10')
('108', 'e8', '20')
('109', 'e9', '10')
('110', 'e10', '20')
('111', 'e11', '10')
('113', 'e13', '10')




#### **[Q #020]. 월급이 200, 400, 2500이 아닌 사원의 사번, 이름, 보너스**

In [27]:
select(text   = 'A #020', table_name = 'employee',
       column = 'eno, ename, commission', cond = 'not salary in (200, 400, 2500)')

select(text = 'A #020 REF', table_name = 'employee',
      column = 'eno, ename, salary', cond = 'not salary in (200, 400, 2500)')

[query] select eno, ename, commission from employee WHERE not salary in (200, 400, 2500);

[A #020]
('101', 'e1', 100)
('105', 'e5', 100)
('106', 'e6', None)
('107', 'e7', 100)
('108', 'e8', None)
('109', 'e9', 100)
('110', 'e10', None)
('111', 'e11', 0)
('112', 'e12', None)
('113', 'e13', 100)


[query] select eno, ename, salary from employee WHERE not salary in (200, 400, 2500);

[A #020 REF]
('101', 'e1', 100)
('105', 'e5', 450)
('106', 'e6', 500)
('107', 'e7', 520)
('108', 'e8', 670)
('109', 'e9', 800)
('110', 'e10', 120)
('111', 'e11', 130)
('112', 'e12', 220)
('113', 'e13', 520)




#### **[Q #021]. 보너스가 NULL인 직원의 이름과 직급.**

In [28]:
select(text   = 'A #021', table_name = 'employee',
       column = 'ename, job', cond = 'commission is Null')

select(text   = 'A #021 REF', table_name = 'employee',
       column = 'ename, job, commission', cond = 'commission is Null')

[query] select ename, job from employee WHERE commission is Null;

[A #021]
('e2', 'chief')
('e4', 'section')
('e6', 'senior')
('e8', 'chief')
('e10', 'section')
('e12', 'senior')


[query] select ename, job, commission from employee WHERE commission is Null;

[A #021 REF]
('e2', 'chief', None)
('e4', 'section', None)
('e6', 'senior', None)
('e8', 'chief', None)
('e10', 'section', None)
('e12', 'senior', None)




In [29]:
conn.close()

# **99. 참고자료**
## **99-1. 도서** 

## **99-2. 논문, 학술지**

## **99-3. 웹 사이트**
- 작업실 | [SQL/실습] 데이터 검색 - 연습문제 풀어보기 1 (문제) [[블로그 링크]](https://reeme.tistory.com/37)

## **99-4. 데이터셋 출처**
